In [1]:
import os
import shutil

import duckdb

In [10]:

duckdb.sql("""
    COPY 
        (SELECT *, replace(title, '/', '_') AS p_title FROM 'data/landing_zone/revenues_per_day.csv') 
    TO 'data/transformed/revenues_per_day' 
    (FORMAT PARQUET, PARTITION_BY (p_title))"""
    )


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [55]:
 movies_details = duckdb.sql("""SELECT 
                            Title,
                            TRY_CAST(try_strptime(Released, '%d %b %Y') AS DATE) as Released,
                            TRY_CAST(replace(Runtime, ' min', '') AS INT) as Runtime,
                            Rated,
                            list_sort(
                                list_transform(
                                    string_split(Director, ','),
                                    s -> trim(s))
                                ) AS Director,
                            list_sort(
                                list_transform(
                                    string_split(Genre, ','),
                                    s -> trim(s))
                                ) AS Genre,
                            list_sort(
                                list_transform(
                                    string_split(Writer, ','),
                                    s -> trim(s))
                                ) AS Writer,
                            list_sort(
                                list_transform(
                                    string_split(Actors, ','),
                                    s -> trim(s))
                                ) AS Actors,
                            list_sort(
                                list_transform(
                                    string_split(Language, ','),
                                    s -> trim(s))
                                ) AS Language,
                            list_sort(
                                list_transform(
                                    string_split(Country, ','),
                                    s -> trim(s))
                                ) AS Country,
                            Plot,
                            TRY_CAST(Metascore AS INT) AS Metascore,
                            TRY_CAST(imdbRating AS DOUBLE) AS imdbRating,
                            TRY_CAST(replace(imdbVotes, ',', '') AS INT) AS imdbVotes,
                            TRY_CAST(replace(replace(BoxOffice, ',', ''), '$','') as INT) as BoxOffice
                            FROM 'data/landing_zone/movies_details.json'""")


duckdb.sql("""
    COPY movies_details 
    TO 'data/transformed/movies_details' 
    (FORMAT PARQUET)""")